# 评估LLM回答质量

我们借助LLM 构建应用程序，包括评估输入，处理输入，以及在呈现结果给用户之前进行最后的结果检查(openAI自己的OpenAI 提供的 Moderation API)。然而，在构建出这样的系统后，我们应如何确知其运行状况呢？更甚者，当我们将其部署并让用户开始使用之后，我们又该如何追踪其表现，发现可能存在的问题，并
持续优化它的回答质量呢？一般可以通过如下两种方式来实现

**其实在这个问题的处理中，我们已经使用了LLM的本身的理解能力，这个是我们应用开发的主要方向**

**目前的LLM已经有了人类智能的语义分析能力了**

In [15]:
# 功能代码，服务于下面的代码片段
import os
from openai import OpenAI

def get_completion(query_txt, temperature=0,is_stream=True):
    client = OpenAI(api_key='sk-8a2970d003e849e1a680a377eb16d22b',
                base_url = "https://dashscope.aliyuncs.com/compatible-mode/v1"
        )
    completion = client.chat.completions.create(
        model="qwen-plus",
        messages=[
            {"role": "user", "content": query_txt},
        ],
        temperature = temperature,
        stream=is_stream,
        # stream_options={"include_usage": True if is_stream else False}
    )

    if not is_stream:
        # print(completion.model_dump_json())
        print(completion.choices[0].message.content)
    else:
        for chunk in completion:
            # print(chunk.model_dump_json())
            if len(chunk.choices) > 0:
                delta = chunk.choices[0].delta
                if delta and delta.content:
                    # print(f"id: {chunk.id}, content: {chunk.choices[0].delta.content}")
                    print(chunk.choices[0].delta.content,end="")

def get_completion_from_messages(messages, model="qwen-max", temperature=0):
    client = OpenAI(api_key='sk-8a2970d003e849e1a680a377eb16d22b',
                base_url = "https://dashscope.aliyuncs.com/compatible-mode/v1"
        )
    completion = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=temperature, # 控制模型输出的随机程度
    )
    # print(str(response.choices[0].message))
    token_dict = {
        'prompt_tokens':completion.usage.prompt_tokens,
        'completion_tokens':completion.usage.completion_tokens,
        'total_tokens':completion.usage.total_tokens,
    }
    print("token_dict: %s" % token_dict)
    # print(inspect.getmembers(completion))
    return completion.choices[0].message.content

## 1. 存在一个简单的正确答案

当应用程序逐渐成熟，测试的重要性也随之增加。通常，当我们仅处理少量样本，手动运行测试并对结果进行评估是可行的。然而，随着开发集的增大，这种方法变得既繁琐又低效。此时，就需要引入自动化测试来提高我们的工作效率。下面将开始编写代码来自动化测试流程，可以帮助您提升效率并确保测试的准确率。

把一些用户问题的标准答案进行搜集整理，用于评估 LLM 回答的准确度，与机器学习中的验证集的作用相当。

1.1 标准测试集

In [51]:
# 标准答案测试集：
msg_ideal_pairs_set = [
# eg 0
{'customer_msg':"""如果我预算有限，我可以买哪种电视？""",
'ideal_answer':{
'电视和家庭影院系统':set(
['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV',
'SoundMax Soundbar', 'CineView OLED TV']
)}
},
# eg 1
{'customer_msg':"""我需要一个智能手机的充电器""",
'ideal_answer':{
'智能手机和配件':set(
['MobiTech PowerCase', 'MobiTech Wireless Charger', 'SmartX EarBuds']
)}
},

# eg 2
{'customer_msg':f"""你有什么样的电脑""",
'ideal_answer':{
'电脑和笔记本':set(
['TechPro 超极本', 'BlueWave 游戏本', 'PowerLite Convertible',
'TechPro Desktop', 'BlueWave Chromebook'
])
}
},
# eg 3
{'customer_msg':f"""告诉我关于smartx pro手机和fotosnap相机的信息，那款DSLR的。\
另外，你们有哪些电视？""",
'ideal_answer':{
'智能手机和配件':set(
['SmartX ProPhone']),
'相机和摄像机':set(
['FotoSnap DSLR Camera']),
'电视和家庭影院系统':set(
['CineView 4K TV', 'SoundMax Home Theater','CineView 8K TV',
'SoundMax Soundbar', 'CineView OLED TV'])
}
},

# eg 4
{'customer_msg':"""告诉我关于CineView电视，那款8K电视、\
Gamesphere游戏机和X游戏机的信息。我的预算有限，你们有哪些电脑？""",
'ideal_answer':{
'电视和家庭影院系统':set(
['CineView 8K TV']),
'游戏机和配件':set(
['GameSphere X']),
'电脑和笔记本':set(
['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLiteConvertible', 'TechPro Desktop', 'BlueWave Chromebook'])
}
},
# eg 5
{'customer_msg':f"""你们有哪些智能手机""",
'ideal_answer':{
'智能手机和配件':set(
['SmartX ProPhone', 'MobiTech PowerCase', 'SmartX MiniPhone','MobiTech Wireless Charger', 'SmartX EarBuds'
])
}
},

# eg 6
{'customer_msg':f"""我预算有限。你能向我推荐一些智能手机吗？""",
'ideal_answer':{
'智能手机和配件':set(
['SmartX EarBuds', 'SmartX MiniPhone', 'MobiTech PowerCase', 'SmartXProPhone', 'MobiTech Wireless Charger']
)}
},
# eg 7 # this will output a subset of the ideal answer
{'customer_msg':f"""有哪些游戏机适合我喜欢赛车游戏的朋友？""",
'ideal_answer':{
'游戏机和配件':set([
'GameSphere X',
'ProGamer Controller',
'GameSphere Y',
'ProGamer Racing Wheel',
'GameSphere VR Headset'
])}
},

# eg 8
{'customer_msg':f"""送给我摄像师朋友什么礼物合适？""",
'ideal_answer': {
'相机和摄像机':set([
'FotoSnap DSLR Camera', 'ActionCam 4K', 'FotoSnap Mirrorless Camera',
'ZoomMaster Camcorder', 'FotoSnap Instant Camera'
])}
},
# eg 9
{'customer_msg':f"""我想要一台热水浴缸时光机""",
'ideal_answer': []
}
]

# print(msg_ideal_pairs_set)

#产品信息
products_and_category ={
'电脑和笔记本': ['TechPro 超极本',
	'BlueWave 游戏本',
	'PowerLite Convertible',
	'TechPro Desktop',
	'BlueWave Chromebook'],
'智能手机和配件': ['SmartX ProPhone'],
'专业手机': ['MobiTech PowerCase',
	'SmartX MiniPhone',
	'MobiTech Wireless Charger',
	'SmartX EarBuds'],
'电视和家庭影院系统': ['CineView 4K TV',
	'SoundMax Home Theater',
	'CineView 8K TV',
	'SoundMax Soundbar',
	'CineView OLED TV'],
'游戏机和配件': ['GameSphere X',
	'ProGamer Controller',
	'GameSphere Y',
	'ProGamer Racing Wheel',
	'GameSphere VR Headset'],
'音频设备': ['AudioPhonic Noise-Canceling Headphones',
	'WaveSound Bluetooth Speaker',
	'AudioPhonic True Wireless Earbuds',
	'WaveSound Soundbar',
	'AudioPhonic Turntable'],
'相机和摄像机': ['FotoSnap DSLR Camera',
	'ActionCam 4K',
	'FotoSnap Mirrorless Camera',
	'ZoomMaster Camcorder',
	'FotoSnap Instant Camera']
}


def find_category_and_product_v2(user_input, products_and_category):
    """
    从用户输入中获取到产品和类别，如果没有对应的产品，则只回答 ['没有找到匹配的产品']
    添加：不要输出任何不符合 JSON 格式的额外文本。
    添加了第二个示例（用于 few-shot 提示），用户询问最便宜的计算机。
    在这两个 few-shot 示例中，显示的响应只是 JSON 格式的完整产品列表。
    参数：
        user_input：用户的查询
        products_and_category：产品类型和对应产品的字典
    """
    delimiter = "####"
    
    # 构建系统消息，指导AI如何解析用户查询
    system_message = f"""
您将提供客户服务查询。

客户服务查询将用{delimiter}字符分隔。

输出一个 Python列表...完整内容和格式见下方详细代码,不要再包含其他内容。

允许的产品：{products_and_category}

"""

    # 示例用户输入和期望的AI响应
    few_shot_user_1 = """我想要最贵的电脑。你推荐哪款？"""
    few_shot_assistant_1 = """
[{'category': '电脑和笔记本', 
  'products': ['TechPro 超极本', 'BlueWave 游戏本', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}]
"""

    few_shot_user_2 = """我想要最便宜的电脑。你推荐哪款？"""
    few_shot_assistant_2 = """
[{'category': '电脑和笔记本', 
  'products': ['TechPro 超极本', 'BlueWave 游戏本', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}]
"""

    # 组装消息列表
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': f"{delimiter}{few_shot_user_1}{delimiter}"},
        {'role': 'assistant', 'content': few_shot_assistant_1},
        {'role': 'user', 'content': f"{delimiter}{few_shot_user_2}{delimiter}"},
        {'role': 'assistant', 'content': few_shot_assistant_2},
        {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"},
    ]
    
    # 调用函数获取结果
    print(f"mesg: {messages}")
    return get_completion_from_messages(messages)

1.2 测试标准答案和LLM的回复

In [ ]:
import json
def eval_response_with_ideal(response,
                          ideal,
                          debug=False):
    """
    评估回复是否与理想答案匹配
    参数：
        response: 回复的内容
        ideal: 理想的答案
        debug: 是否打印调试信息
    """
    if debug:
        print("回复：")
        print(response)

    # json.loads() 只能解析双引号，因此此处将单引号替换为双引号
    json_like_str = response.replace("'", '"')

    # 解析为一系列的字典
    # print(json_like_str)
    try:
        l_of_d = json.loads(json_like_str)
    except json.JSONDecodeError:
        print("json.loads() failed")
        return 0

    # 当响应为空，即没有找到任何商品时
    if l_of_d == [] and ideal == []:
        return 1

    # 另外一种异常情况是，标准答案数量与回复答案数量不匹配
    elif l_of_d == [] or ideal == []:
        return 0

    # 统计正确答案数量
    correct = 0

    if debug:
        print("l_of_d is")
        print(l_of_d)

    # 对每一个问答对
    for d in l_of_d:
        # 获取产品和目录
        # print(f"respone：{d}")
        # continue
        cat = d.get('category')
        prod_l = d.get('products')

        # 有获取到产品和目录
        if cat and prod_l:
            # convert list to set for comparison
            prod_set = set(prod_l)

            # get ideal set of products
            ideal_cat = ideal.get(cat)

            if ideal_cat:
                prod_set_ideal = set(ideal.get(cat))
            else:
                if debug:
                    print(f"没有在标准答案中找到目录 {cat}")
                    print(f"标准答案: {ideal}")
                continue

            if debug:
                print("产品集合：\n", prod_set)
                print()
                print("标准答案的产品集合：\n", prod_set_ideal)

            # 查找到的产品集合和标准的产品集合一致
            if prod_set == prod_set_ideal:
                if debug:
                    print("正确")
                correct += 1
            else:
                print("错误")
                print(f"产品集合: {prod_set}")
                print(f"标准的产品集合: {prod_set_ideal}")

                if prod_set <= prod_set_ideal:
                    print("回答是标准答案的一个子集")
                elif prod_set >= prod_set_ideal:
                    print("回答是标准答案的一个超集")

    # 计算正确答案数
    pc_correct = correct / len(l_of_d)
    return pc_correct

print(f'用戶提問：{msg_ideal_pairs_set[0]["customer_msg"]}')
print(f'标准答案：{msg_ideal_pairs_set[0]["ideal_answer"]}')

score_accum = 0
for i, pair in enumerate(msg_ideal_pairs_set):
	print(f"示例 {i}")
	# if i < 3:
	# 	continue
	customer_msg = pair['customer_msg']
	ideal = pair['ideal_answer']
	print("Customer message",customer_msg)
	print("ideal:",ideal)
	response = find_category_and_product_v2(customer_msg,products_and_category)
	# print("products_by_category",products_by_category)
	score = eval_response_with_ideal(response,ideal,debug=True)
	print(f"{i}: {score}")
	score_accum += score

n_examples = len(msg_ideal_pairs_set)
fraction_correct = score_accum / n_examples
print(f"正确比例为 {n_examples}: {fraction_correct}")

## 2 不存在简单的正确答案

我们如何对一些没有标注答案的回复进行评估：
* 文科的参考答案
* LLM的生产的回答

**两个设计模式方案**

* 即使没有专家提供的理想答案，只要能制定一个评估标准，就可以使用一个 LLM 来评估另一个
LLM 的输出。
* 如果您可以提供一个专家提供的理想答案，那么可以帮助您的 LLM 更好地比较特定助手输出是否与
专家提供的理想答案相似。



In [ ]:

assistant_answer = """
关于SmartX Pro手机和FotoSnap DSLR相机的信息：
1. SmartX Pro手机（型号：SX-PP10）是一款功能强大的智能手机，拥有6.1英寸显示屏、128GB存储空
间、12MP双摄像头和5G网络支持。价格为899.99美元，保修期为1年。
2. FotoSnap DSLR相机（型号：FS-DSLR200）是一款多功能的单反相机，拥有24.2MP传感器、1080p视
频拍摄、3英寸液晶屏和可更换镜头。价格为599.99美元，保修期为1年。
关于电视的信息：
我们有以下电视可供选择：
1. CineView 4K电视（型号：CV-4K55）- 55英寸显示屏，4K分辨率，支持HDR和智能电视功能。价格为
599.99美元，保修期为2年。
2. CineView 8K电视（型号：CV-8K65）- 65英寸显示屏，8K分辨率，支持HDR和智能电视功能。价格为
2999.99美元，保修期为2年。
3. CineView OLED电视（型号：CV-OLED55）- 55英寸OLED显示屏，4K分辨率，支持HDR和智能电视功
能。价格为1499.99美元，保修期为2年。
请问您对以上产品有任何特别的要求或其他问题吗？
"""


# 用户消息
customer_msg = f"""
告诉我有关 the smartx pro phone 和 the fotosnap camera, the dslr one 的信息。
另外，你们这有什么 TVs ？"""

# 问题、上下文
cust_prod_info = {
    'customer_msg': customer_msg,
    'context': products_and_category
}

def eval_with_rubric(test_set, assistant_answer):
    """
    使用 GPT API 评估生成的回答
    参数：
        test_set: 测试集
        assistant_answer: 助手的回复
    """
    
    # 提取测试集关键数据
    cust_msg = test_set['customer_msg']
    context = test_set['context']
    completion = assistant_answer

    # 构建系统消息 - 定义评估角色
    system_message = """\
你是一位助理，通过查看客户服务代理使用的上下文来评估客户服务代理回答用户问题的情况。
"""

    # 创建用户消息 - 定义评估标准
    user_message = f"""\
你正在根据代理使用的上下文评估对问题的提交答案。以下是数据：
[开始]
************
[用户问题]: {cust_msg}
************
[使用的上下文]: {context}
************
[客户代理的回答]: {completion}
************
[结束]

请将提交的答案的事实内容与上下文进行比较，忽略样式、语法或标点符号上的差异。

回答以下问题：
助手的回应是否只基于所提供的上下文？（是或否）
回答中是否包含上下文中未提供的信息？（是或否）
回应与上下文之间是否存在任何不一致之处？（是或否）

计算用户提出了多少个问题。（输出一个数字）
对于用户提出的每个问题，是否有相应的回答？
问题1：（是或否）
问题2：（是或否）
...
问题N：（是或否）

在提出的问题数量中，有多少个问题在回答中得到了回应？（输出一个数字）"""

    # 组装消息列表并获取评估结果
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]
    
    response = get_completion_from_messages(messages)
    return response

# 执行评估并打印结果
evaluation_output = eval_with_rubric(cust_prod_info, assistant_answer)
print(evaluation_output)

In [ ]:
'''基于中文Prompt的验证集'''
test_set_ideal = {
'customer_msg': """\
告诉我有关 the Smartx Pro 手机 和 FotoSnap DSLR相机, the dslr one 的信息。\n另外，你们这
有什么电视 ？""",
'ideal_answer':"""\
SmartX Pro手机是一款功能强大的智能手机，拥有6.1英寸显示屏、128GB存储空间、12MP双摄像头和5G网
络支持。价格为899.99美元，保修期为1年。
FotoSnap DSLR相机是一款多功能的单反相机，拥有24.2MP传感器、1080p视频拍摄、3英寸液晶屏和可更
换镜头。价格为599.99美元，保修期为1年。
我们有以下电视可供选择：
1. CineView 4K电视（型号：CV-4K55）- 55英寸显示屏，4K分辨率，支持HDR和智能电视功能。价格为
599.99美元，保修期为2年。
2. CineView 8K电视（型号：CV-8K65）- 65英寸显示屏，8K分辨率，支持HDR和智能电视功能。价格为
2999.99美元，保修期为2年。
3. CineView OLED电视（型号：CV-OLED55）- 55英寸OLED显示屏，4K分辨率，支持HDR和智能电视功
能。价格为1499.99美元，保修期为2年。
"""
}

def eval_vs_ideal(test_set, assistant_answer):
    """
    评估回复是否与理想答案匹配
    参数：
        test_set: 测试集
        assistant_answer: 助手的回复
    """
    
    # 提取测试集关键数据
    cust_msg = test_set['customer_msg']
    ideal = test_set['ideal_answer']
    completion = assistant_answer

    # 构建系统消息模板
    system_message = """\
您是一位助理，通过将客户服务代理的回答与理想（专家）回答进行比较，
评估客户服务代理对用户问题的回答质量。
请输出一个单独的字母（A、B、C、D、E），不要包含其他内容。"""

    # 创建用户消息模板
    user_message = f"""\
您正在比较一个给定问题的提交答案和专家答案。数据如下:
[开始]
************
[问题]: {cust_msg}
************
[专家答案]: {ideal}
************
[提交答案]: {completion}
************
[结束]

比较提交答案的事实内容与专家答案，关注在内容上，忽略样式、语法或标点符号上的差异。
你的关注核心应该是答案的内容是否正确，内容的细微差异是可以接受的。
提交的答案可能是专家答案的子集、超集，或者与之冲突。确定适用的情况，并通过选择以下选项之一回
答问题：
（A）提交的答案是专家答案的子集，并且与之完全一致。
（B）提交的答案是专家答案的超集，并且与之完全一致。
（C）提交的答案包含与专家答案完全相同的细节。
（D）提交的答案与专家答案存在分歧。
（E）答案存在差异，但从事实的角度来看这些差异并不重要。
选项：ABCDE"""

    # 生成评估提示并获取结果
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]
    
    text_str = json.dumps(messages, ensure_ascii=False,indent=4)
    print(f"request: \n{text_str}")
    print(f"[问题]: {cust_msg}")
    print(f"[专家答案]: {ideal}")
    print(f"[提交答案]: {completion}")
    response = get_completion_from_messages(messages)
    return response


eval_vs_ideal(test_set_ideal, assistant_answer)

# assistant_answer_2 = "life is like a box of chocolates"
# eval_vs_ideal(test_set_ideal, assistant_answer_2)